In [1]:
import pathlib
import sys

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import tqdm

sys.path.append("../helpers")

## load data

In [2]:
# load data
cls_file_path = pathlib.Path(
    "../../1.scDINO_analysis/1.scDINO_run/outputdir/mnist_photos/CLS_features/channel_binary_model_dino_deitsmall16_pretrain_full_checkpoint_features.csv"
).resolve(strict=True)

image_paths_file_path = pathlib.Path(
    "../../1.scDINO_analysis/1.scDINO_run/outputdir/mnist_photos/CLS_features/image_paths.csv"
).resolve(strict=True)

# load in the image paths
image_paths = pd.read_csv(image_paths_file_path, header=None)
print(image_paths.shape)

# load in the the data to a csv
cls_features = pd.read_csv(cls_file_path, header=None)
print(cls_features.shape)
cls_features.head()

(1500000, 1)
(1500000, 384)


,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,-0.029546,-0.093779,0.035240,-0.029979,0.094636,0.060090,-0.011613,-0.011853,0.025654,0.051191,...,0.016053,0.055923,-0.015797,0.009030,0.016759,0.004438,0.090052,0.084351,0.031153,0.027775
1,-0.029816,-0.066288,0.014838,-0.033962,0.069943,0.035519,-0.034434,-0.038205,0.018258,0.085273,...,0.024541,0.068040,-0.034506,0.004123,0.047106,-0.011689,0.088033,0.080813,0.039639,0.052613
2,-0.020296,0.022372,-0.021896,-0.049170,0.018540,-0.052099,-0.106442,0.001228,0.093113,0.024022,...,0.011164,0.067288,0.048709,-0.005579,-0.033400,0.029852,0.000440,0.026421,-0.011651,0.063168
3,-0.020296,0.022372,-0.021896,-0.049170,0.018540,-0.052099,-0.106442,0.001228,0.093113,0.024022,...,0.011164,0.067288,0.048709,-0.005579,-0.033400,0.029852,0.000440,0.026421,-0.011651,0.063168
4,-0.003252,0.024647,0.005446,-0.039569,0.053363,-0.028918,-0.091161,-0.037333,0.029517,0.046394,...,0.024138,0.059754,-0.002547,-0.005610,-0.011385,0.026064,0.008469,0.067664,-0.016873,0.057681


In [3]:
# rename columns
cls_features.columns = [f"CLS_{i}" for i in range(cls_features.shape[1])]
cls_features.head()

,CLS_0,CLS_1,CLS_2,CLS_3,CLS_4,CLS_5,CLS_6,CLS_7,CLS_8,CLS_9,...,CLS_374,CLS_375,CLS_376,CLS_377,CLS_378,CLS_379,CLS_380,CLS_381,CLS_382,CLS_383
0,-0.029546,-0.093779,0.035240,-0.029979,0.094636,0.060090,-0.011613,-0.011853,0.025654,0.051191,...,0.016053,0.055923,-0.015797,0.009030,0.016759,0.004438,0.090052,0.084351,0.031153,0.027775
1,-0.029816,-0.066288,0.014838,-0.033962,0.069943,0.035519,-0.034434,-0.038205,0.018258,0.085273,...,0.024541,0.068040,-0.034506,0.004123,0.047106,-0.011689,0.088033,0.080813,0.039639,0.052613
2,-0.020296,0.022372,-0.021896,-0.049170,0.018540,-0.052099,-0.106442,0.001228,0.093113,0.024022,...,0.011164,0.067288,0.048709,-0.005579,-0.033400,0.029852,0.000440,0.026421,-0.011651,0.063168
3,-0.020296,0.022372,-0.021896,-0.049170,0.018540,-0.052099,-0.106442,0.001228,0.093113,0.024022,...,0.011164,0.067288,0.048709,-0.005579,-0.033400,0.029852,0.000440,0.026421,-0.011651,0.063168
4,-0.003252,0.024647,0.005446,-0.039569,0.053363,-0.028918,-0.091161,-0.037333,0.029517,0.046394,...,0.024138,0.059754,-0.002547,-0.005610,-0.011385,0.026064,0.008469,0.067664,-0.016873,0.057681


In [4]:
# rename the image paths columns
image_paths.columns = ["Metadata_image_paths"]
# make metadata columns for the image paths
cls_features["Metadata_label"] = image_paths["Metadata_image_paths"].apply(
    lambda x: pathlib.Path(x).stem.split("_")[1]
)
cls_features["Metadata_cell_idx"] = image_paths["Metadata_image_paths"].apply(
    lambda x: pathlib.Path(x).stem.split("_")[3]
)
cls_features["Metadata_Time"] = image_paths["Metadata_image_paths"].apply(
    lambda x: pathlib.Path(x).stem.split("_")[5]
)
# reorder the columns so that the metadata columns are first
cls_features = cls_features[
    ["Metadata_label", "Metadata_cell_idx", "Metadata_Time"]
    + cls_features.columns[:-3].tolist()
]

# make all columns floats
cls_features = cls_features.astype(float)
cls_features.head()

,Metadata_label,Metadata_cell_idx,Metadata_Time,CLS_0,CLS_1,CLS_2,CLS_3,CLS_4,CLS_5,CLS_6,...,CLS_374,CLS_375,CLS_376,CLS_377,CLS_378,CLS_379,CLS_380,CLS_381,CLS_382,CLS_383
0,5.0,10006.0,0.0,-0.029546,-0.093779,0.035240,-0.029979,0.094636,0.060090,-0.011613,...,0.016053,0.055923,-0.015797,0.009030,0.016759,0.004438,0.090052,0.084351,0.031153,0.027775
1,5.0,10006.0,1.0,-0.029816,-0.066288,0.014838,-0.033962,0.069943,0.035519,-0.034434,...,0.024541,0.068040,-0.034506,0.004123,0.047106,-0.011689,0.088033,0.080813,0.039639,0.052613
2,5.0,10006.0,10.0,-0.020296,0.022372,-0.021896,-0.049170,0.018540,-0.052099,-0.106442,...,0.011164,0.067288,0.048709,-0.005579,-0.033400,0.029852,0.000440,0.026421,-0.011651,0.063168
3,5.0,10006.0,11.0,-0.020296,0.022372,-0.021896,-0.049170,0.018540,-0.052099,-0.106442,...,0.011164,0.067288,0.048709,-0.005579,-0.033400,0.029852,0.000440,0.026421,-0.011651,0.063168
4,5.0,10006.0,12.0,-0.003252,0.024647,0.005446,-0.039569,0.053363,-0.028918,-0.091161,...,0.024138,0.059754,-0.002547,-0.005610,-0.011385,0.026064,0.008469,0.067664,-0.016873,0.057681


In [5]:
cls_tensor = torch.tensor(cls_features.iloc[:, 3:].values)
# reshape the data from (cell_idx, time, features) to (cell_idx, features*time)
print(cls_tensor.shape)
cls_tensor = torch.tensor(cls_features.iloc[:, 3:].values)
cls_tensor = cls_tensor.reshape(
    -1, cls_features.Metadata_Time.nunique(), cls_tensor.shape[1]
)

print(cls_tensor.shape)

torch.Size([1500000, 384])
torch.Size([60000, 25, 384])


In [6]:
import torch.nn

# make a Dataset
import torch.utils


class CLSDataset(torch.utils.data.Dataset):
    def __init__(self, cls_features: pd.DataFrame):
        super(CLSDataset, self).__init__()
        self.cls_features = cls_features

    def __len__(self):
        return self.cls_features.shape[0]

    @staticmethod
    def mask_timepoints(tensor: torch.Tensor, timepoints: int):
        # mask one timepoint in the tensor
        # random value from 0 to timepoints
        random_timepoint = np.random.randint(0, timepoints)
        tensor[random_timepoint, :] = 1
        return tensor

    def __getitem__(self, idx):
        y = self.cls_features[idx, :, :]
        x = self.mask_timepoints(y, y.shape[0])
        return x, y


# make a DataLoader
cls_dataset = CLSDataset(cls_tensor)
cls_loader = torch.utils.data.DataLoader(cls_dataset, batch_size=20, shuffle=False)
# get the first batch size
x, y = next(iter(cls_loader))
x.shape, y.shape

(torch.Size([20, 25, 384]), torch.Size([20, 25, 384]))

## Define the model

Code adapted from https://medium.com/correll-lab/building-a-vision-transformer-model-from-scratch-a3054f707cc6

In [7]:
class PositionWiseEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super().__init__()

        self.cls_token = nn.Parameter(
            torch.randn(1, 1, d_model)
        )  # Classification Token

        # Creating positional encoding
        pe = torch.zeros(max_seq_length + 1, d_model)

        for pos in range(max_seq_length):
            for i in range(d_model):
                if i % 2 == 0:
                    pe[pos][i] = torch.sin(
                        pos / 10000 ** (2 * i / torch.tensor(d_model))
                    )
                else:
                    pe[pos][i] = torch.cos(
                        pos / 10000 ** (2 * i / torch.tensor(d_model))
                    )

        self.register_buffer("pe", pe.unsqueeze(0))

    def forward(self, x):
        # Expand to have class token for every image in batch
        tokens_batch = self.cls_token.expand(x.size()[0], -1, -1)

        # Adding class tokens to the beginning of each embedding
        x = torch.cat((tokens_batch, x), dim=1)

        # Add positional encoding to embeddings
        x = x + self.pe[:, : x.size(1), :]

        return x


# test the PositionWiseEncoding
d_model = torch.tensor(512)
max_seq_length = torch.tensor(10)
pe = PositionWiseEncoding(d_model, max_seq_length)
pe.forward(torch.randn(2, 10, 512))

/tmp/ipykernel_1422998/3781259385.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pos / 10000 ** (2 * i / torch.tensor(d_model))
/tmp/ipykernel_1422998/3781259385.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pos / 10000 ** (2 * i / torch.tensor(d_model))


tensor([[[-0.8520,  1.5166,  0.2156,  ...,  1.1039, -1.4358,  0.5494],
         [-0.3311, -0.7266,  0.8207,  ..., -0.4056, -0.3778,  1.9836],
         [ 0.8693, -0.0301,  0.8754,  ...,  0.8945, -0.3249,  0.6853],
         ...,
         [ 0.5532,  1.2587,  0.2955,  ...,  2.1656, -0.4710,  0.0662],
         [ 1.2258,  0.7680, -0.2650,  ...,  0.1283,  0.5580,  0.5760],
         [ 1.7366,  0.2619, -0.2478,  ...,  1.4762, -0.4329, -0.7781]],

        [[-0.8520,  1.5166,  0.2156,  ...,  1.1039, -1.4358,  0.5494],
         [-0.5047,  1.4123,  1.7736,  ...,  3.2340,  0.5619,  1.1527],
         [ 1.1611, -0.7600, -2.4386,  ...,  3.1187,  0.8189,  3.4872],
         ...,
         [ 0.7750, -0.3959,  0.0059,  ...,  1.8952, -0.5980,  0.8607],
         [-0.6186, -2.8388,  0.6824,  ..., -0.4546, -0.8062, -0.7220],
         [-0.2989, -0.1119, -0.1014,  ...,  2.0180,  0.4916,  0.3415]]],
       grad_fn=<AddBackward0>)

In [8]:
class AttentionHead(nn.Module):
    def __init__(self, d_model, head_size):
        super().__init__()
        self.head_size = head_size

        self.query = nn.Linear(d_model, head_size)
        self.key = nn.Linear(d_model, head_size)
        self.value = nn.Linear(d_model, head_size)

    def forward(self, x):
        # Obtaining Queries, Keys, and Values
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)

        # Dot Product of Queries and Keys
        attention = Q @ K.transpose(-2, -1)

        # Scaling
        attention = attention / (self.head_size**0.5)

        attention = torch.softmax(attention, dim=-1)

        attention = attention @ V

        return attention


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.head_size = d_model // n_heads

        self.W_o = nn.Linear(d_model, d_model)

        self.heads = nn.ModuleList(
            [AttentionHead(d_model, self.head_size) for _ in range(n_heads)]
        )

    def forward(self, x):
        # Combine attention heads
        out = torch.cat([head(x) for head in self.heads], dim=-1)

        out = self.W_o(out)

        return out


# test the MultiHeadAttention
d_model = torch.tensor(512)
n_heads = torch.tensor(8)
mha = MultiHeadAttention(d_model, n_heads)
mha.forward(torch.randn(2, 10, 512))

tensor([[[-0.2291,  0.1659,  0.0525,  ...,  0.1605, -0.0658,  0.0952],
         [-0.1823,  0.1036,  0.0826,  ...,  0.1731, -0.0188,  0.0398],
         [-0.1861,  0.1028,  0.0512,  ...,  0.1523, -0.0127,  0.0357],
         ...,
         [-0.2443,  0.0841,  0.0678,  ...,  0.1335, -0.0594,  0.0773],
         [-0.1849,  0.0549,  0.0582,  ...,  0.1972, -0.1228,  0.1137],
         [-0.2479,  0.0503,  0.0762,  ...,  0.1961, -0.1098,  0.0595]],

        [[ 0.3021,  0.2776, -0.1754,  ...,  0.1452, -0.0092, -0.0438],
         [ 0.2084,  0.1718, -0.1744,  ...,  0.1735, -0.1209, -0.0229],
         [ 0.2806,  0.2008, -0.1458,  ...,  0.1322,  0.0151,  0.0187],
         ...,
         [ 0.2828,  0.2494, -0.1780,  ...,  0.0848, -0.0567, -0.0185],
         [ 0.2985,  0.2268, -0.1691,  ...,  0.0582, -0.0411, -0.0832],
         [ 0.2444,  0.2720, -0.1666,  ...,  0.1067,  0.0453,  0.0076]]],
       grad_fn=<ViewBackward0>)

In [9]:
class TransformerEncoder(nn.Module):
    def __init__(self, d_model, n_heads, r_mlp=4):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads

        # Sub-Layer 1 Normalization
        self.ln1 = nn.LayerNorm(d_model)

        # Multi-Head Attention
        self.mha = MultiHeadAttention(d_model, n_heads)

        # Sub-Layer 2 Normalization
        self.ln2 = nn.LayerNorm(d_model)

        # Multilayer Perception
        self.mlp = nn.Sequential(
            nn.Linear(d_model, d_model * r_mlp),
            nn.GELU(),
            nn.Linear(d_model * r_mlp, d_model),
        )

    def forward(self, x):
        # Residual Connection After Sub-Layer 1
        out = x + self.mha(self.ln1(x))

        # Residual Connection After Sub-Layer 2
        out = out + self.mlp(self.ln2(out))

        return out


# test the TransformerEncoder
d_model = 384
n_heads = 8
r_mlp = 4
te = TransformerEncoder(d_model, n_heads, r_mlp)
te.forward(torch.randn(2, 10, 384, dtype=torch.float32))

tensor([[[ 1.3648, -0.1755,  0.4912,  ...,  1.0293,  1.1186,  1.1497],
         [ 0.3566, -1.5364,  0.3265,  ..., -0.0484, -0.8855, -0.4042],
         [-0.6996,  1.4297,  0.1730,  ..., -0.8811, -0.4816,  3.0337],
         ...,
         [-0.4999,  0.1597,  0.8121,  ..., -0.0100,  0.1191, -0.3411],
         [-1.3076,  0.4992, -0.5681,  ..., -1.3836, -0.3848, -1.2969],
         [ 0.3112, -0.7330,  0.3809,  ..., -0.1236,  0.3230,  0.4748]],

        [[-0.0469,  1.4134,  1.6866,  ...,  1.4317, -0.2477,  1.6590],
         [-1.2577, -1.3425, -0.6988,  ...,  0.2587, -1.0133,  0.2153],
         [-0.3154,  0.5306,  1.4096,  ...,  0.3559,  0.8954, -2.0120],
         ...,
         [ 1.6707, -1.3565, -0.4847,  ..., -1.4779,  0.0339, -2.5156],
         [ 0.6411,  0.3048, -0.2843,  ..., -1.1491,  1.7195, -0.4898],
         [-0.2715, -1.1387, -0.9141,  ..., -1.0430, -0.0797, -0.0657]]],
       grad_fn=<AddBackward0>)

In [10]:
class TemporalTransformer(nn.Module):
    def __init__(self, d_model, n_classes, n_heads, n_layers, seq_len):
        super().__init__()

        assert d_model % n_heads == 0, "d_model must be divisible by n_heads"

        self.d_model = d_model  # Dimensionality of model
        self.n_classes = n_classes  # Number of classes
        self.n_heads = n_heads  # Number of attention heads
        self.seq_len = seq_len  # Sequence length

        self.positional_encoding = PositionWiseEncoding(self.d_model, self.seq_len)
        self.transformer_encoder = nn.Sequential(
            *[TransformerEncoder(self.d_model, self.n_heads) for _ in range(n_layers)]
        )

        # Classification MLP
        self.classifier = nn.Sequential(
            nn.Linear(self.d_model, self.n_classes), nn.Softmax(dim=-1)
        )

    def forward(self, x):

        x = self.positional_encoding(x)

        x = self.transformer_encoder(x)

        # x = self.classifier(x[:,0])
        # remove the class token
        x = x[:, 1:, :]

        return x


# test the Transformer
d_model = 384
n_classes = 10
n_heads = 8
n_layers = 4
seq_len = 10
tt = TemporalTransformer(d_model, n_classes, n_heads, n_layers, seq_len)
tt.forward(torch.randn(2, 10, 384))

tensor([[[ 1.0514,  1.1926,  1.1813,  ..., -0.0934,  0.1782,  1.6328],
         [ 0.6155, -1.1070,  0.0936,  ...,  0.5635,  0.6225, -1.0014],
         [ 1.1637,  0.1411,  0.7319,  ..., -2.5400, -0.5850, -0.0731],
         ...,
         [ 1.9988,  1.7400,  0.4203,  ...,  1.1801, -0.6749, -0.9886],
         [ 1.6551, -0.4430,  0.1328,  ...,  1.1269, -0.4040,  0.7960],
         [ 0.0944, -1.2211,  0.0543,  ...,  0.1675,  0.3382, -1.0189]],

        [[ 1.5985,  2.0573,  1.6586,  ..., -1.7370, -1.4980,  0.7284],
         [ 1.7054,  0.8217, -0.3673,  ...,  0.4460, -0.0578,  1.9114],
         [-0.3797, -1.0880,  0.5592,  ...,  0.1785,  0.2280, -0.0547],
         ...,
         [ 2.2847,  1.9717, -0.6047,  ...,  0.4749, -1.1031, -0.1852],
         [ 2.2427, -0.9110, -0.8853,  ..., -2.0369,  1.4158,  1.3443],
         [-0.0526,  0.6650, -0.0909,  ..., -1.1365, -0.9365,  0.2878]]],
       grad_fn=<SliceBackward0>)

## Test to see if the model works?

In [17]:
d_model = 384
n_classes = 10
n_heads = 12
n_layers = 8
batch_size = 32
epochs = 50
alpha = 0.05

seq_len = 25
# training data loader is cls_loader

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(
    "Using device: ",
    device,
    f"({torch.cuda.get_device_name(device)})" if torch.cuda.is_available() else "",
)

transformer = TemporalTransformer(d_model, n_classes, n_heads, n_layers, seq_len).to(
    device
)
# import adam
from torch.optim import Adam

optimizer = Adam(transformer.parameters(), lr=alpha)
criterion = nn.MSELoss()

for epoch in range(epochs):

    training_loss = 0.0
    for i, data in enumerate(cls_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device).float(), labels.to(device).float()
        optimizer.zero_grad()
        outputs = transformer(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        training_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs} loss: {training_loss  / len(cls_loader) :.3f}")

Using device:  cuda (NVIDIA GeForce RTX 3090 Ti)
Epoch 1/50 loss: 15785.364
Epoch 2/50 loss: 9205543.888
Epoch 3/50 loss: 13335215.535
Epoch 4/50 loss: 125900.951
Epoch 5/50 loss: 8607415.818
Epoch 6/50 loss: 139964.030
Epoch 7/50 loss: 3268688.919
Epoch 8/50 loss: 2133496.189
Epoch 9/50 loss: 5145497.964
Epoch 10/50 loss: 68212.372
Epoch 11/50 loss: 87717.813
Epoch 12/50 loss: 7763610.521
Epoch 13/50 loss: 150699.345
Epoch 14/50 loss: 5725287.957
Epoch 15/50 loss: 677967.750
Epoch 16/50 loss: 1463432.424
Epoch 17/50 loss: 25734.189
Epoch 18/50 loss: 4526396.255
Epoch 19/50 loss: 74826.618
Epoch 20/50 loss: 1526481.728
Epoch 21/50 loss: 16807.412
Epoch 22/50 loss: 28945.068
Epoch 23/50 loss: 4443634.065
Epoch 24/50 loss: 82033.652
Epoch 25/50 loss: 2695550.868
Epoch 26/50 loss: 198415.600
Epoch 27/50 loss: 137859.083
Epoch 28/50 loss: 50971.955
Epoch 29/50 loss: 1297717.178
Epoch 30/50 loss: 54474.891
Epoch 31/50 loss: 1677355.889
Epoch 32/50 loss: 50343.792
Epoch 33/50 loss: 34976.386

In [20]:
list_of_reconstructed = []
list_of_original = []
with torch.no_grad():
    for data in cls_loader:
        x, y = data
        x, y = x.to(device).float(), y.to(device).float()

        # reshape (B, T, d) -> (B*T, d)
        x = transformer(x)
        x = x.reshape(-1, x.shape[-1])
        y = y.reshape(-1, y.shape[-1])
        list_of_reconstructed.append(x.cpu().numpy())
        list_of_original.append(y.cpu().numpy())

In [21]:
# make two different dfs for the reconstructed and original data
reconstructed_df = pd.DataFrame(np.concatenate(list_of_reconstructed))
original_df = pd.DataFrame(np.concatenate(list_of_original))

# add the metadata columns
reconstructed_df = pd.concat([cls_features.iloc[:, :3], reconstructed_df], axis=1)
original_df = pd.concat([cls_features.iloc[:, :3], original_df], axis=1)

# add label for reconstructed and original
reconstructed_df["Metadata_reconstructed"] = "reconstructed_df"
original_df["Metadata_reconstructed"] = "original_df"

# combine the two dataframes
combined_df = pd.concat([reconstructed_df, original_df])
combined_df.head()
# rename columns if int columns
combined_df.rename(
    columns={
        col: f"CLS_{col}" if isinstance(col, int) else col
        for col in combined_df.columns
    },
    inplace=True,
)
# metadata columns
metadata_columns = [col for col in combined_df.columns if "Metadata" in col]
combined_df.reset_index(drop=True, inplace=True)
features_df = combined_df.drop(metadata_columns, axis=1)

features_df.head()

,CLS_0,CLS_1,CLS_2,CLS_3,CLS_4,CLS_5,CLS_6,CLS_7,CLS_8,CLS_9,...,CLS_374,CLS_375,CLS_376,CLS_377,CLS_378,CLS_379,CLS_380,CLS_381,CLS_382,CLS_383
0,51.759766,645.414062,-275.111816,-30.500000,-329.777344,239.705078,350.839844,119.054688,-2704.781250,267.146484,...,145.539062,504.338196,432.873047,285.309082,202.203979,242.397461,-533.054688,-419.226562,666.085938,-150.983398
1,51.878906,644.570312,-274.915527,-31.322266,-329.529785,239.039062,351.185547,118.449219,-2704.484375,266.621094,...,145.570312,504.352936,432.884766,285.304810,202.225220,242.434570,-533.062500,-419.242188,666.074219,-150.958984
2,51.208984,643.980469,-275.475586,-32.025391,-329.935547,238.355469,350.964844,117.761719,-2704.609375,265.980469,...,145.611328,504.379883,432.906250,285.311890,202.214233,242.489258,-533.078125,-419.242188,666.089844,-150.937500
3,50.384766,644.187500,-276.349609,-32.126953,-330.697266,238.052734,350.333984,117.320312,-2705.031250,265.476562,...,145.650391,504.393127,432.925781,285.305176,202.216858,242.533203,-533.070312,-419.230469,666.082031,-150.903320
4,50.189453,644.988281,-276.873535,-31.544922,-331.369629,238.320312,349.587891,117.367188,-2705.671875,265.353516,...,145.642578,504.394104,432.910156,285.307373,202.191589,242.540039,-533.070312,-419.214844,666.089844,-150.915039


In [22]:
# umap
import umap

reducer = umap.UMAP(n_neighbors=15, n_components=2, metric="euclidean")
embedding = reducer.fit_transform(features_df)
embedding_df = pd.DataFrame(embedding, columns=["UMAP1", "UMAP2"])
embedding_df["Metadata_reconstructed"] = combined_df["Metadata_reconstructed"]
embedding_df["Metadata_label"] = combined_df["Metadata_label"]
embedding_df["Metadata_cell_idx"] = combined_df["Metadata_cell_idx"]
embedding_df.head()

/home/lippincm/miniforge3/envs/transformers_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/lippincm/miniforge3/envs/transformers_env/lib/python3.12/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


In [ ]:
# plot the umap
import matplotlib.pyplot as plt
import seaborn as sns

# randomize the rows for plotting
embedding_df = embedding_df.sample(frac=1)
# two subplots
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
sns.scatterplot(
    data=embedding_df, x="UMAP1", y="UMAP2", hue="Metadata_reconstructed", ax=ax[0]
)
sns.scatterplot(data=embedding_df, x="UMAP1", y="UMAP2", hue="Metadata_label", ax=ax[1])
plt.show()